In [1]:
from model import make_model as CNN

import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# question 1: The best loss function?
# answer : binaryCrossEntropyloss()


In [3]:
print(CNN().summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 175232)            0         
                                                                 
 dense (Dense)               (None, 64)                11214912  
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 11,215,873
Trainable params: 11,215,873
Non-trainable params: 0
____________________________________________

In [4]:
# Question 2: total_parameters :-> 11, 215, 873

# Generators and Training

In [5]:
def generator(train, **kwargs):
    gen_ = ImageDataGenerator(**kwargs)
    if train:
        return gen_.flow_from_directory("./train",
                                       target_size = (150, 150),
                                       batch_size = 20, class_mode = "binary")
    return gen_.flow_from_directory("./validation", target_size = (150, 150),
                                   batch_size = 20, class_mode = "binary")

In [6]:
def train_model(model, train_generator, validation_generator, aug=False):
    history = model.fit(
    train_generator,
        steps_per_epoch = 100,
        epochs = 10,
        validation_data = validation_generator,
        validation_steps = 50,
    )
    if aug:
        return np.mean(history.history["val_loss"]), history.history["val_accuracy"]
    return np.median(history.history["accuracy"]), np.std(history.history["loss"])

In [7]:
train_generator = generator(train = True, rescale=1./255)
validation_generator = generator(train = False, rescale=1./255)


Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [8]:
model = CNN()
optimizer = keras.optimizers.SGD(learning_rate = 2e-3, momentum = 8e-1)
loss_fn = keras.losses.BinaryCrossentropy()
    
model.compile(optimizer = optimizer, loss=loss_fn, metrics = ["accuracy"])

In [9]:
tr_med_accu, tr_std_loss = train_model(model, train_generator, validation_generator, aug=False)

Epoch 1/10
100/100 [==============================] - 24s 237ms/step - loss: 0.7043 - accuracy: 0.5230 - val_loss: 0.6821 - val_accuracy: 0.5600
Epoch 2/10
100/100 [==============================] - 24s 238ms/step - loss: 0.6838 - accuracy: 0.5630 - val_loss: 0.6787 - val_accuracy: 0.5740
Epoch 3/10
100/100 [==============================] - 28s 281ms/step - loss: 0.6761 - accuracy: 0.5725 - val_loss: 0.6707 - val_accuracy: 0.5760
Epoch 4/10
100/100 [==============================] - 28s 283ms/step - loss: 0.6665 - accuracy: 0.5870 - val_loss: 0.6727 - val_accuracy: 0.5670
Epoch 5/10
100/100 [==============================] - 28s 284ms/step - loss: 0.6666 - accuracy: 0.5895 - val_loss: 0.6637 - val_accuracy: 0.5860
Epoch 6/10
100/100 [==============================] - 27s 267ms/step - loss: 0.6575 - accuracy: 0.5995 - val_loss: 0.6740 - val_accuracy: 0.5600
Epoch 7/10
100/100 [==============================] - 31s 306ms/step - loss: 0.6439 - accuracy: 0.6135 - val_loss: 0.6595 - val_ac

In [10]:
# Question 3: median of training accuracy?
print(f"Median training accuracy: {tr_med_accu}")

# question 4: standard deviation of training loss?
print(f"Traning loss standard deviation: {tr_std_loss}")

Median training accuracy: 0.5945000052452087
Traning loss standard deviation: 0.020492336742692758


In [11]:
extra_kwargs = {
         "rotation_range": 40,
        "width_shift_range": 0.2,
        "height_shift_range": 0.2,
        "shear_range": 0.2,
         "zoom_range": 0.2,
        "horizontal_flip": True,
         "fill_mode": 'nearest',
}

In [12]:
train_generator = generator(train=True, **extra_kwargs)
validation_generator = generator(train=False, **extra_kwargs)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [13]:
mean_val_loss, val_accuracy = train_model(model, train_generator, validation_generator, aug=True)

Epoch 1/10
100/100 [==============================] - 33s 325ms/step - loss: 37.8040 - accuracy: 0.4995 - val_loss: 0.6979 - val_accuracy: 0.4810
Epoch 2/10
100/100 [==============================] - 35s 348ms/step - loss: 0.6937 - accuracy: 0.4945 - val_loss: 0.6943 - val_accuracy: 0.4970
Epoch 3/10
100/100 [==============================] - 35s 352ms/step - loss: 0.6927 - accuracy: 0.5160 - val_loss: 0.6944 - val_accuracy: 0.5030
Epoch 4/10
100/100 [==============================] - 33s 324ms/step - loss: 0.6929 - accuracy: 0.5130 - val_loss: 0.6945 - val_accuracy: 0.4740
Epoch 5/10
100/100 [==============================] - 34s 336ms/step - loss: 0.6940 - accuracy: 0.4815 - val_loss: 0.6940 - val_accuracy: 0.4580
Epoch 6/10
100/100 [==============================] - 33s 334ms/step - loss: 0.6932 - accuracy: 0.4990 - val_loss: 0.6932 - val_accuracy: 0.4900
Epoch 7/10
100/100 [==============================] - 32s 318ms/step - loss: 0.6931 - accuracy: 0.5125 - val_loss: 0.6938 - val_a

In [14]:
# Question 5: validation mean loss after augmentation?
print(f"Median validation loss: {mean_val_loss}")

Median validation loss: 0.6941949903964997


In [18]:
# question 6: average of validation accuracy for last 5 epochs
print(np.mean(val_accuracy[5:10]))

0.48780000805854795
